In [15]:
from glob import glob
from astropy.io import fits
import numpy as np
from ccdproc import ImageFileCollection
import matplotlib.pyplot as plt

### 0. Verify all FITS headers, as some do not conform to standard.

This was a very frustrating step. On November 2 and 3, the FITS headers had two entries for LST and HA. This broke my normal file I/O code, which would have taken me a long time to rewrite. So, I wrote this small script to remove those parameters from the FITS headers for all of my files. Hopefully, this issue can be fixed at the telescope-level.

Also, I manually went in and changed all forward slashes in FITS filenames to periods.

In [2]:
def standardize_FITS(filepath):
    all_filenames = glob(filepath, recursive=True)
    n = 0
    for filename in all_filenames:
        data, header = fits.getdata(filename, header=True)
        header.remove('LST', ignore_missing=True)
        header.remove('LST', ignore_missing=True) # Need to do this twice... because there's two of them
        header.remove('HA', ignore_missing=True)
        header.remove('HA', ignore_missing=True) # Need to do this twice... because there's two of them
        fits.writeto(filename, data, header, overwrite=True)
        n += 1
        print(n,"done")
    print("all done!")

In [3]:
# standardize_FITS('data/**/*.fits') # Standardize all FITS files for the project

### 0.1. Import CY Aqr frames, dark frames, and flat frames, excluding flagged bad frames.

In [5]:
directory_day1 = 'data/2020-10-26'
ic_cyaqr_day1 = ImageFileCollection(directory_day1, keywords='*', glob_include='CY_Aqr*', glob_exclude='*bad*')
ic_darks_day1 = ImageFileCollection(directory_day1, keywords='*', glob_include='dark*', glob_exclude='*bad*')
directory_day2 = 'data/2020-10-27'
ic_cyaqr_day2 = ImageFileCollection(directory_day2, keywords='*', glob_include='CY_Aqr*', glob_exclude='*bad*')
ic_darks_day2 = ImageFileCollection(directory_day2, keywords='*', glob_include='dark*', glob_exclude='*bad*')
ic_flats_day2 = ImageFileCollection(directory_day2, keywords='*', glob_include='flat*', glob_exclude='*bad*')
directory_day3 = 'data/2020-11-02'
ic_cyaqr_day3 = ImageFileCollection(directory_day3, keywords='*', glob_include='CY_Aqr*', glob_exclude='*bad*')
ic_darks_day3 = ImageFileCollection(directory_day3, keywords='*', glob_include='dark*', glob_exclude='*bad*')
ic_flats_day3 = ImageFileCollection(directory_day3, keywords='*', glob_include='flat*', glob_exclude='*bad*')
directory_day4 = 'data/2020-11-03'
ic_cyaqr_day4 = ImageFileCollection(directory_day4, keywords='*', glob_include='CY_Aqr*', glob_exclude='*bad*')
ic_darks_day4 = ImageFileCollection(directory_day4, keywords='*', glob_include='dark*', glob_exclude='*bad*')
ic_flats_day4 = ImageFileCollection(directory_day4, keywords='*', glob_include='flat*', glob_exclude='*bad*')

### 0.2. Reduce the data.
First, I'll produce **final dark frames** for each science and flat exposure time.

In [30]:
def make_final_dark(ic_files, directory):
    # Pass this function some dark frames from an image collection
    # and it will create a final dark frame.
    lst = []
    for i in range(len(ic_files)):
        lst.append(fits.getdata(directory+'/'+ic_files[i]))
    arr = np.array(lst)
    final_dark = np.median(arr, axis=0)
    return final_dark

In [132]:
final_dark_15s_day1_day2 = make_final_dark(ic_darks_day1.files_filtered(exptime=15), directory_day1)
final_dark_5s_day1_day2 = make_final_dark(ic_darks_day2.files_filtered(exptime=5), directory_day2)
final_dark_15s_day3 = make_final_dark(ic_darks_day3.files_filtered(exptime=15), directory_day3)
final_dark_2s_day3 = make_final_dark(ic_darks_day3.files_filtered(exptime=2), directory_day3)
final_dark_15s_day4 = make_final_dark(ic_darks_day4.files_filtered(exptime=15), directory_day4)
final_dark_2s_day4 = make_final_dark(ic_darks_day4.files_filtered(exptime=2), directory_day4)

Then, I'll produce **normalized final flat frames** for each filter (only V band).

In [133]:
def make_norm_final_flat(ic_files, directory, final_dark):
    # Pass this function some flat frames and a final dark frame
    # and it will create a normalized final flat field image.
    lst = []
    for i in range(len(ic_files)):
        lst.append(fits.getdata(directory+'/'+ic_files[i]))
    arr = np.array(lst)
    median_combined_flat = np.median(arr, axis=0)
    final_flat = median_combined_flat - final_dark # These should have the same exposure time
    norm_final_flat = final_flat/np.nanmedian(final_flat)
    return norm_final_flat

In [134]:
norm_final_flat_day1_day2 = make_norm_final_flat(ic_flats_day2.files_filtered(exptime=5), directory_day2, final_dark_5s_day1_day2)
norm_final_flat_day3 = make_norm_final_flat(ic_flats_day3.files_filtered(exptime=2), directory_day3, final_dark_2s_day3)
norm_final_flat_day4 = make_norm_final_flat(ic_flats_day4.files_filtered(exptime=2), directory_day4, final_dark_2s_day4)

Now, I will perform dark-subtraction and flat-field-correction on our science frames in the V band. Since we are only working with a single exposure time, this process is simplified. First, I'll dark-subtract each individual science frame using the **final dark frame** with the correct exposure time. I'll also keep the time of the observation linked with  each science frame.

In [135]:
def get_star_images_times(ic_files, directory):
    frames = []
    times = []
    for i in range(len(ic_files)):
        frame = fits.getdata(directory+'/'+ic_files[i])
        frames.append(frame)
        hdr = fits.getheader(directory+'/'+ic_files[i])
        times.append(hdr['DATE-OBS'])
    frames_arr = np.array(frames)
    times_arr = np.array(times)
    return frames_arr, times_arr

In [136]:
# Extract images of stars as well as exposure times
cyaqr_im_day1, cyaqr_tm_day1 = get_star_images_times(ic_cyaqr_day1.files_filtered(exptime=15), directory_day1)
cyaqr_im_day2, cyaqr_tm_day2 = get_star_images_times(ic_cyaqr_day2.files_filtered(exptime=15), directory_day2)
cyaqr_im_day3, cyaqr_tm_day3 = get_star_images_times(ic_cyaqr_day3.files_filtered(exptime=15), directory_day3)
cyaqr_im_day4, cyaqr_tm_day4 = get_star_images_times(ic_cyaqr_day4.files_filtered(exptime=15), directory_day4)

In [137]:
# Use array broadcasting to dark-subtract appropriate dark frames from each science frame
cyaqr_im_day1_d = cyaqr_im_day1 - final_dark_15s_day1_day2[np.newaxis,:,:] # Use array brdcstng to dark-subtract
cyaqr_im_day2_d = cyaqr_im_day2 - final_dark_15s_day1_day2[np.newaxis,:,:] # Use array brdcstng to dark-subtract
cyaqr_im_day3_d = cyaqr_im_day3 - final_dark_15s_day3[np.newaxis,:,:] # Use array brdcstng to dark-subtract
cyaqr_im_day4_d = cyaqr_im_day4 - final_dark_15s_day4[np.newaxis,:,:] # Use array brdcstng to dark-subtract

Then, I'll flat-field correct each individual science frame using the **final flat frame** for the correct filter.

In [138]:
cyaqr_im_day1_d_f = cyaqr_im_day1_d/norm_final_flat_day1_day2[np.newaxis,:,:] # Arr brdcstng to flat-field correct
cyaqr_im_day2_d_f = cyaqr_im_day2_d/norm_final_flat_day1_day2[np.newaxis,:,:] # Arr brdcstng to flat-field correct
cyaqr_im_day3_d_f = cyaqr_im_day3_d/norm_final_flat_day3[np.newaxis,:,:] # Arr brdcstng to flat-field correct
cyaqr_im_day4_d_f = cyaqr_im_day4_d/norm_final_flat_day4[np.newaxis,:,:] # Arr brdcstng to flat-field correct